In [1]:
import os
import json

def get_data(folder_name):
    x = []
    y = []
    positions = []
    file_names = []

    for file in os.listdir(folder_name):
        if file.endswith(".txt"):
            file_name = os.path.join(folder_name, file[:-4])

            file_text = open(file_name + '.txt', encoding='utf8')
            try:
                file_truth = open(file_name + '.truth', encoding='utf8')

                try:
                    text = file_text.read()
                    truth = json.load(file_truth)
                    truth_changes = truth['changes']
                    truth_positions = truth['positions']

                    x.append(text)
                    y.append(truth_changes)
                    positions.append(truth_positions)
                    file_names.append(file[:-4])
                finally:
                    file_truth.close()
            finally:
                file_text.close()

    return x, y, positions, file_names

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [3]:
TRAINING_DIR = '../../data/training'
X, y, positions, file_names = get_data(TRAINING_DIR)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [6]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

top_words = 10000

tokenizer = Tokenizer(lower=True, split=" ")
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [7]:
max_doc_length = 2000
X_train = sequence.pad_sequences(X_train, maxlen=max_doc_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_doc_length)

In [8]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('../../data/external/embeddings/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [9]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [10]:
# create the model
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_doc_length))
model.add(LSTM(100, dropout=0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 100)         4612200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 4,692,701
Trainable params: 4,692,701
Non-trainable params: 0
_________________________________________________________________
None
Train on 1996 samples, validate on 984 samples
Epoch 1/15
1996/1996 [==============================] - 121s 60ms/step - loss: 0.7026 - acc: 0.5020 - val_loss: 0.6943 - val_acc: 0.5020
Epoch 2/15
1996/1996 [==============================] - 147s 73ms/step - loss: 0.6928 - acc: 0.5356 - val_loss: 0.6925 - val_acc: 0.5203
Epoch 3/15
1996/1996 [==============================] - 166s 83ms/step

In [11]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 51.63%
